# Project1

## Step 1
Dependencies and Setup

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json

from scipy.stats import linregress

from api_keys import geoapify_key

In [2]:
# Load the CSV file
df=pd.read_csv("./Resources/Kaggle/Student Loan Debt by School 2020-2021.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42740 entries, 0 to 42739
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  42740 non-null  int64  
 1   OPE ID                 42740 non-null  int64  
 2   School                 42740 non-null  object 
 3   City                   33650 non-null  object 
 4   State                  40700 non-null  object 
 5   Zip Code               33650 non-null  float64
 6   School Type            42740 non-null  object 
 7   Loan Type              42740 non-null  object 
 8   Recipients             42740 non-null  float64
 9   # of Loans Originated  42740 non-null  float64
 10  $ of Loans Originated  42740 non-null  float64
 11  # of Disbursements     42740 non-null  float64
 12  $ of Disbursements     42740 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 4.2+ MB


In [3]:
# Change data type and rename columns
df['Zip Code'] = df['Zip Code'].fillna(-1).astype(int).astype(str).replace('-1', 'NaN')
df['Recipients'] = df['Recipients'].astype(int)
df['# of Loans Originated'] = df['# of Loans Originated'].astype(int)
df['# of Disbursements'] = df['# of Disbursements'].astype(int)
df=df[df['Loan Type']!='Total']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42740 entries, 0 to 42739
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  42740 non-null  int64  
 1   OPE ID                 42740 non-null  int64  
 2   School                 42740 non-null  object 
 3   City                   33650 non-null  object 
 4   State                  40700 non-null  object 
 5   Zip Code               42740 non-null  object 
 6   School Type            42740 non-null  object 
 7   Loan Type              42740 non-null  object 
 8   Recipients             42740 non-null  int32  
 9   # of Loans Originated  42740 non-null  int32  
 10  $ of Loans Originated  42740 non-null  float64
 11  # of Disbursements     42740 non-null  int32  
 12  $ of Disbursements     42740 non-null  float64
dtypes: float64(2), int32(3), int64(2), object(6)
memory usage: 3.8+ MB


In [4]:
# df.head(10)
df.tail(10)

,index,OPE ID,School,City,State,Zip Code,School Type,Loan Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
42730,42730,42919,FLORIDA PROFESSIONAL INSTITUTE,MIAMI,FL,33155,Proprietary,Total,20,20,66672.0,20,33336.0
42731,42731,34165,DALLAS NURSING INSTITUTE,NaN,TX,NaN,Private-Nonprofit,Total,20,20,54000.0,20,54000.0
42732,42732,21265,ENID BEAUTY COLLEGE,NaN,OK,NaN,Proprietary,Total,20,20,52000.0,20,26000.0
42733,42733,42907,PREMIERE AESTHETICS INSTITUTE,BEND,OR,97702,Proprietary,Total,100,100,304292.0,100,152124.0
42734,42734,41869,"INTERNATIONAL COLLEGE OF BEAUTY, ARTS & SCIENCES",LOS ANGELES,CA,90039,Proprietary,Total,12,12,15996.0,12,7992.0
42735,42735,42377,REGIONAL CENTER FOR BORDER HEALTH,SOMERTON,AZ,85350,Private-Nonprofit,Total,40,40,68672.0,80,68672.0
42736,42736,40293,TRENDSETTERS OF FL. SCHOOL OF BEAUTY & BARBERING,JACKSONVILLE,FL,32205,Proprietary,Total,40,40,51012.0,80,51012.0
42737,42737,4666,AMERICAN COLLEGE FOR MEDICAL CAREERS,ORLANDO,FL,32809,Proprietary,Total,20,20,9000.0,20,4504.0
42738,42738,42932,PROFESSIONAL COSMETOLOGY ACADEMY (THE),LAREDO,TX,78041,Proprietary,Total,8,8,23336.0,8,11664.0
42739,42739,42185,INNOVATE SALON ACADEMY,SOUTH PLAINFIELD,NJ,7080,Proprietary,Total,40,40,40008.0,80,40008.0


In [5]:
df.describe()

,index,OPE ID,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
count,42740.000000,42740.000000,42740.000000,42740.000000,4.274000e+04,42740.000000,4.274000e+04
mean,21369.500000,16201.447824,3798.421385,3909.581844,3.379861e+07,3974.820660,1.694132e+07
std,12338.119589,15479.494190,16547.929427,17095.803201,1.666734e+08,17393.746307,8.551740e+07
min,0.000000,1002.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
25%,10684.750000,2848.000000,6.000000,7.000000,3.383300e+04,7.000000,1.700000e+04
50%,21369.500000,8447.500000,130.000000,132.000000,8.019415e+05,137.000000,3.912990e+05
75%,32054.250000,30837.000000,1287.000000,1318.250000,8.253579e+06,1350.000000,4.020021e+06
max,42739.000000,42933.000000,776959.000000,798927.000000,7.191430e+09,798927.000000,3.629214e+09


## Step 2
### Ask the questions

State, What is the breakdown of number of collages in each state? P

State, What is the breakdown of recipients of Student Loans by State (by quantity and by $)?P

State, What states have the most (public schools? Private schools?) recipients?P

State, What are the top 10 states with the most student loans?L

Geographic, How does the average student loan vary by zip code?M

Geographic, Are there trends based on geographic region or type of school?T

In [6]:
# get US, none zero data
drop_df= df[(df['Zip Code']!='NaN')&(df['Zip Code']!='0')&(df['Recipients']!=0)]

# drop_df.to_csv('./Output/test.csv')
drop_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28261 entries, 0 to 42739
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  28261 non-null  int64  
 1   OPE ID                 28261 non-null  int64  
 2   School                 28261 non-null  object 
 3   City                   28261 non-null  object 
 4   State                  28261 non-null  object 
 5   Zip Code               28261 non-null  object 
 6   School Type            28261 non-null  object 
 7   Loan Type              28261 non-null  object 
 8   Recipients             28261 non-null  int32  
 9   # of Loans Originated  28261 non-null  int32  
 10  $ of Loans Originated  28261 non-null  float64
 11  # of Disbursements     28261 non-null  int32  
 12  $ of Disbursements     28261 non-null  float64
dtypes: float64(2), int32(3), int64(2), object(6)
memory usage: 2.7+ MB


In [7]:
zip_df=pd.read_csv("./Resources/Zipcode/uszips.csv")
# zip_df.info()
zip_df['Full Zip Code']= zip_df['Zip Code'].astype(str).str.zfill(5)
zip_df['Zip Code']=zip_df['Zip Code'].astype(str)
zip_df.tail(10)
# zip_df['Zip Code']=zip_df['Zip Code'].astype(str)
# drop_LL_df=pd.merge(drop_df,zip_df,how="left",on='Zip Code')
# drop_LL_df



,Zip Code,Lat,Lng,City,State,Full Zip Code
33899,37933,NaN,NaN,KNOXVILLE,TN,37933
33900,27108,NaN,NaN,WINSTON-SALEM,NC,27108
33901,80160,NaN,NaN,LITTLETON,CO,80160
33902,78806,NaN,NaN,BATON ROUGE,LA,78806
33903,1086,NaN,NaN,WESTFIELD,MA,01086
33904,28035,NaN,NaN,DAVIDSON,NC,28035
33905,29721,NaN,NaN,LANCASTER,SC,29721
33906,29802,NaN,NaN,AIKEN,SC,29802
33907,681,NaN,NaN,MAYAGUEZ,PR,00681
33908,614,NaN,NaN,ARECIBO,PR,00614


In [8]:
# create lat and lng on the dataframe
# drop_LL_df['Full Zip Code']= drop_LL_df['Zip Code'].str.zfill(5)

# drop_LL_df.to_csv('./Output/withoutLL.csv')
# drop_LL_df.info()

In [12]:
# it takes too long to run all

base_url='https://api.geoapify.com/v1/geocode/search?'
params = {
    
    "apiKey": geoapify_key,
    "type":"postcode",
    'format':'json',
    'country':'USA'
}


for index, row in zip_df.iterrows():

    params["text"] = row['Full Zip Code']
    params["city"] = row['City']
    params["state"] = row['State']
    if pd.isnull(row['Lat']):

        LL = requests.get(base_url, params=params)

    # Convert the API response to JSON format
        LL = LL.json()
            
        try:
            print(row['Full Zip Code'])         
            print(LL["results"][0]["lat"])
            print(LL["results"][0]["lon"])
            zip_df.loc[index, "Lat"] = LL["results"][0]["lat"]
            zip_df.loc[index, "Lng"] = LL["results"][0]["lon"]
        except (KeyError, IndexError):
            params.pop("text")
            params.pop("type")
            LL = requests.get(base_url, params=params)
            # print(params)
            LL = LL.json()
            # print(json.dumps(LL,indent=4))
            zip_df.loc[index, "Lat"] = LL["results"][0]["lat"]
            zip_df.loc[index, "Lng"] = LL["results"][0]["lon"]
zip_df.to_csv('./Output/zipwithLL.csv',index=False)


35294
33.50208462
-86.80429502
35762
{'apiKey': '8a0afac814b24ccbb3c98c22623479d2', 'format': 'json', 'country': 'USA', 'city': 'NORMAL', 'state': 'AL'}
{
    "results": [
        {
            "datasource": {
                "sourcename": "openstreetmap",
                "attribution": "\u00a9 OpenStreetMap contributors",
                "license": "Open Database License",
                "url": "https://www.openstreetmap.org/copyright"
            },
            "name": "Normal",
            "country": "United States",
            "country_code": "us",
            "state": "Alabama",
            "county": "Madison County",
            "city": "Huntsville",
            "hamlet": "Normal",
            "postcode": "35810",
            "lon": -86.5719369,
            "lat": 34.7889792,
            "state_code": "AL",
            "formatted": "Normal, Huntsville, AL 35810, United States of America",
            "address_line1": "Normal",
            "address_line2": "Huntsville, AL 35810,

In [ ]:
# # drop_LL_df.info()
# drop_NoneLL_df=drop_LL_df[pd.isnull(drop_LL_df['Lat'])]
# drop_NoneLL_df

Loan Type, What is the breakdown of type of student loan  (subsidized, unsubsidized, grad plus, parent plus, etc.)?M

School Type, What is the breakdown of student loan quantity/$ by school type (for profit, non-profit, public, etc.)?L

<!-- VS, What is the breakdown in disbursements versus loan origination $ amounts? -->

<!-- VS, Which schools have the highest disparity between loans originated and loans disbursed? -->

API, Foreign university, VS US

More datasets